In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import probfit
import iminuit
from scipy.stats import norm

from probfit import UnbinnedLH
from probfit import gaussian
from iminuit import Minuit



In [ ]:
original_df1=pd.read_hdf("/mnt/t3nfs01/data01/shome/micheli/energyRegProject_20180223/CMSSW_8_0_4/src/RegressionTraining/applyRegression/ntuples_NN/Ntup_10Nov_Photon_testing.hd5")
original_df_EE=pd.read_hdf("/mnt/t3nfs01/data01/shome/micheli/energyRegProject_20180223/CMSSW_8_0_4/src/RegressionTraining/applyRegression/ntuples_NN_EE/Ntup_10Nov_Photon_testing_EE.hd5")
print("finished")

In [ ]:
def plotting(x_plot,y_plot,y_err_fit,y_err_sigma):
    plt.figure()
    plt.ylim(0.95,1.1)
    plt.title("Profile $response$ vs ${}$, Fit error".format(column))
    plt.ylabel(r"$response$")
    plt.xlabel(r"${}$".format(column))
    plt.errorbar(x_plot,y_plot,y_err_fit,fmt='b.')
    plt.ylim([0.9,1.1])
    plt.savefig("plots/profile_plots_response/Fit_Error_{}response_vs_{}".format(suffix,column))
    plt.show()
    plt.close()
    plt.figure()
    plt.title("Profile $response$ vs ${}$, Sigma error".format(column))
    plt.ylabel(r"$response$")
    plt.xlabel(r"${}$".format(column))
    plt.ylim([0.9,1.1])
    plt.errorbar(x_plot,y_plot,y_err_sigma,fmt='b.')
    plt.savefig("plots/profile_plots_response/Sigma_Error_{}response_vs_{}".format(suffix,column))
    plt.show()
    plt.close()


In [ ]:
k=0

for original_df in [original_df1,original_df_EE]:#

    if k==0:
        suffix='NN'
        k+=1
    else:
        suffix='NN_EE'
    
    for column in ['pt','scEta','rhoValue']:
        lower_bound=0.4
        upper_bound=1.2
        amountbins=20
        
        y_plot=np.zeros(amountbins)
        x_plot=np.zeros(amountbins)
        y_err_fit=np.zeros(amountbins)
        y_err_sigma=np.zeros(amountbins)
        df=original_df.copy()

        df.loc[:,'response']=np.reciprocal(df.loc[:,'response'].values)
        df=df[df['response']>lower_bound]
        df=df[df['response']<upper_bound]
        if column=='pt':
            df=df[df[column]<3000]
        df.loc[:,column]=df[abs(df[column]-df[column].mean())<3*df[column].std()][column] #outlier fuck up binning otherwise
        notused,bins=pd.cut(df[column],amountbins, retbins=True)   
        
        for i in xrange(0,len(bins)-1):
            try:
                
                #selecting data in bin
                df_temp=df[df[column] >=bins[i]]
                df_temp=df_temp[df_temp[column]<=bins[i+1]]
                #outlier control, sometimes there are responses up to 25 which fucks the binning 
                data=df_temp.loc[:,'response'].astype(float).values #abs(df_temp['response']-df_temp['response'].mean())<3*df_temp['response'].std()]
                
                #skip if interval is near empty
                print "size={}".format(len(data))
                #if len(data)<.01*len(df['response'])/amountbins:
                  #  continue
                
                #900 bins for gauss, equidistant
                n,bins_gauss,patches=plt.hist(data,bins=[j*0.001 for j in xrange(0,1200)],normed=True)
                plt.title("histogramm $response$ for ${}$ in {} to {}".format(column,int(10*bins[i])/10.,int(10*bins[i+1])/10.))
                if np.sum(n) ==0:
                    continue
                plt.savefig("plots/CB_{}/Histogramm{}_{}".format(column,suffix,i)) 
                plt.show()
    
                plt.close()
                
                
                
                # set the boudaryof the fit 
                bound_gauss=[bins_gauss[np.argmax(n)-3],bins_gauss[np.argmax(n)+3]]#centered around the max
                #maybe smooth the line by averaging n of 5 neighbors for a more reliant argmax 
                # always remember to normalize the pdf in the boundary
                normalized_gaussian = probfit.Normalized(probfit.gaussian, bound_gauss)
                # build the likelihood with the previous bound
                binned_likelihood = probfit.BinnedLH(normalized_gaussian, data, bins=20, bound=bound_gauss)
                pars_gauss = dict(mean  = bins_gauss[np.argmax(n)],
                                  sigma = 0.05,
                                  limit_mean=(0.99,1.01)) 
                
                # do the fit
                minuit = iminuit.Minuit(binned_likelihood, **pars_gauss)
                minuit.migrad()      

                # print results
                print(minuit.values)
                print(minuit.errors)

                # get the fit values to be used as imput later
                mean_gauss  = minuit.values[0]
                sigma_gauss = minuit.values[1]
                # print mu_gauss, sigma_gauss
    
                # plot the results
                binned_likelihood.draw(minuit)
                plt.show()
                plt.close()
                plt.close()
                # now fit the cb using as starting point the gaussian mean and sigma

                # define the fit range to get the long left tail of the CB and just enough on the right to get the core
                bound_cb=[mean_gauss-3*sigma_gauss, mean_gauss+1*sigma_gauss]
                # always remember to normalize the pdf in the boundary
                normalized_crystalball = probfit.Normalized(probfit.crystalball, bound_cb)

                # build the likelihood with the previous bound
                binned_likelihood_cb   = probfit.BinnedLH(normalized_crystalball, data, bins=100, bound=bound_cb)
                pars_cb = dict(mean  = mean_gauss,  
                               sigma = sigma_gauss,
                               alpha = 0.5,  
                               n     = 3.0, 
                               fix_sigma=True,
                               fix_mean=True
                               )

                # do the fit
            
                # do the fit
                minuit = iminuit.Minuit(binned_likelihood_cb, **pars_cb)
                minuit.migrad()    
                
                ax=plt.subplot2grid((6,3),(0,0), rowspan=4,colspan=4)   
                plt.title("Crystalball $response$ for ${}$ in {} to {}".format(column,int(10*bins[i])/10.,int(10*bins[i+1])/10.))


                (data_edges, datay), (errorp, errorm), (total_pdf_x, total_pdf_y),parts=binned_likelihood_cb.draw(minuit,parmloc=((0.7,0.95)),nfbins=101)
                coeff=minuit.args
                print coeff
                x_plot[i]=df_temp[column].values[np.argmin(df_temp['response'].values-coeff[2])]
                y_plot[i]=coeff[2]
                y_err_fit[i]+=minuit.errors[2]
                y_err_sigma[i]+=coeff[3]
                ratio=datay/total_pdf_y
                plt.subplot2grid((5,3),(4,0),rowspan=2,colspan=4).set_yticks([0.8,1])

                
                plt.plot((data_edges[1:]+data_edges[:-1])/2,ratio,'bo')
                plt.ylabel(r"$ratio$")
                plt.xlabel("response")


                plt.savefig("plots/CB_{}/{}_{}".format(column,suffix,i)) 
                
                plt.show()
                plt.close()
            except Exception as e:
                print e
                print "erroor"
        plotting(x_plot,y_plot,y_err_fit,y_err_sigma)


In [ ]:
bins_gauss